In [1]:
# Task 4: QSVM-like variational classifier on Iris (binary)
import pennylane as qml
from pennylane import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Load binary Iris (setosa vs versicolor)
iris = datasets.load_iris()
X = iris.data
y = iris.target
# choose classes 0 and 1 only
mask = y < 2
X = X[mask][:, :2]  # pick first two features for simplicity (2 features -> 2 qubits)
y = y[mask]
y = (y == 1).astype(int)  # 0 or 1

In [3]:
# Preprocess
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

In [4]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, random_state=42)

In [5]:
# Device
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

# Proposal A: angle-encoding + shallow variational layers
def feature_map_A(x):
    # x has length 2
    for i in range(n_qubits):
        qml.RX(x[i], wires=i)

def variational_layer_A(params):
    # params shape (n_qubits,)
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
    # entangle
    qml.CNOT(wires=[0,1])

In [6]:
@qml.qnode(dev)
def circuit_A(x, params):
    feature_map_A(x)
    variational_layer_A(params)
    return qml.expval(qml.PauliZ(0))  # use expectation of Z on wire 0 as decision 

# Prediction helper
def predict_A(X, params):
    preds = []
    for x in X:
        out = circuit_A(x, params)
        preds.append(0 if out > 0 else 1)
    return np.array(preds)

In [13]:
# Train A (simple gradient descent)
init_params_A = np.random.randn(n_qubits) * 0.1
opt = qml.GradientDescentOptimizer(stepsize=0.2)
params_A = init_params_A.copy()
epochs = 50
for ep in range(epochs):
    def loss(p):
        preds = [circuit_A(x, p) for x in X_train]
        # map expectation (in [-1,1]) to probabilities: use (1 - exp)/2 as prob of label 1
        probs = [(1 - val) / 2 for val in preds]
        # binary cross-entropy
        loss_val = -np.mean([y_train[i] * np.log(max(probs[i],1e-8)) + (1-y_train[i]) * np.log(max(1-probs[i],1e-8)) for i in range(len(y_train))])
        return float(loss_val)
    params_A = opt.step(loss, params_A)
    if ep % 10 == 0:
        print(f"Epoch {ep}, loss {loss(params_A):.4f}")
acc_A_train = np.mean(predict_A(X_train, params_A) == y_train)
acc_A_test = np.mean(predict_A(X_test, params_A) == y_test)
print("Proposal A - train acc:", acc_A_train, "test acc:", acc_A_test)

ValueError: setting an array element with a sequence.

In [14]:
# Proposal B: feature-map with 2-qubit interaction + deeper variational layers
layers = 2
def feature_map_B(x):
    # encode non-linear features via entangling rotations
    qml.RZ(x[0], wires=0)
    qml.RZ(x[1], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RX(x[0]*x[1], wires=0)

def variational_layer_B(params, layer):
    # params has shape (layers, n_qubits, 2) for RY and RZ
    for i in range(n_qubits):
        qml.RY(params[layer,i,0], wires=i)
        qml.RZ(params[layer,i,1], wires=i)
    # entanglement ring
    qml.CNOT(wires=[0,1])

In [15]:
@qml.qnode(dev)
def circuit_B(x, params):
    feature_map_B(x)
    for l in range(params.shape[0]):
        variational_layer_B(params, l)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# predict by combining expectations
def predict_B(X, params):
    preds = []
    for x in X:
        exps = circuit_B(x, params)
        # simple combiner
        score = exps[0] - exps[1]
        preds.append(0 if score > 0 else 1)
    return np.array(preds)

In [16]:
# initialize params
params_B = np.random.randn(layers, n_qubits, 2) * 0.1
optB = qml.AdamOptimizer(0.1)
epochs = 60
for ep in range(epochs):
    def lossB(p):
        preds = [predict_B([x], p)[0] for x in X_train]  # not differentiable; instead use expectation-based loss
        # better: use raw expectations as logits
        outs = [circuit_B(x, p) for x in X_train]
        # map to score and use MSE with labels
        scores = [out[0] - out[1] for out in outs]
        return np.mean((np.array(scores) - (1-2*y_train))**2)
    params_B = optB.step(lossB, params_B)
    if ep % 10 == 0:
        print(f"EpochB {ep}, loss {lossB(params_B):.4f}")

acc_B_train = np.mean(predict_B(X_train, params_B) == y_train)
acc_B_test = np.mean(predict_B(X_test, params_B) == y_test)
print("Proposal B - train acc:", acc_B_train, "test acc:", acc_B_test)

EpochB 0, loss 1.0807
EpochB 10, loss 0.9918
EpochB 20, loss 0.9829
EpochB 30, loss 0.9759
EpochB 40, loss 0.9754
EpochB 50, loss 0.9752
Proposal B - train acc: 0.6285714285714286 test acc: 0.4666666666666667


In [17]:
# Quick expressibility probe: sample random params and compute pairwise state fidelities
def expressibility_probe(n_samples=50):
    rand_params = np.random.randn(n_samples, layers, n_qubits, 2)
    states = []
    for p in rand_params:
        # get statevector
        @qml.qnode(dev)
        def s():
            feature_map_B([0.1, 0.2])  # irrelevant static input
            for l in range(p.shape[0]):
                variational_layer_B(p, l)
            return qml.state()
        states.append(s())
    # compute pairwise fidelities
    F = []
    for i in range(len(states)):
        for j in range(i+1, len(states)):
            F.append(np.abs(np.vdot(states[i], states[j]))**2)
    F = np.array(F)
    print("Expressibility probe -> mean fidelity:", np.mean(F), "std:", np.std(F))
    return np.mean(F), np.std(F)

In [18]:
if __name__ == "__main__":
    expressibility_probe()

Expressibility probe -> mean fidelity: 0.3593606964319812 std: 0.23233766980746454
